In [ ]:
from snowflake.snowpark import Session
import json as js
import requests
import pandas as pd
from requests.exceptions import RequestException, HTTPError

# Get active Snowflake session
session = get_active_session()

# Fetch client ID and secret from Snowflake secure function
try:
    result = session.sql("SELECT TEST_SCHEMA.GET_SECRET_VAL()").collect()
    raw_result = result[0][0]  # Extract the first value of the first row
    parsed_result = raw_result.strip('()').split(',')  # Remove parentheses and split by comma

    # Extract and clean values
    cl_id = parsed_result[0].strip().replace("'", '')  # Clean raw client ID string
    client_secret = parsed_result[1].strip().replace("'", '') 
  # Clean raw client secret string
except Exception as e:
    raise RuntimeError(f"Failed to fetch or parse client credentials: {e}")

# Microsoft OAuth2 Token URL
url = "https://login.microsoftonline.com/<your_tenant_id>/oauth2/v2.0/token"

# Prepare payload for token request
payload = f"grant_type=client_credentials&scope=https://graph.microsoft.com/.default&client_id={cl_id}&client_secret={client_secret}"
headers = {"Content-Type": "application/x-www-form-urlencoded"}
# Microsoft Oauth2 authentication
try:
    response = requests.get(url, data=payload, headers=headers)
    response.raise_for_status()  # Raise HTTPError for bad responses
    token_js = js.loads(response.text)
    token = token_js["access_token"]
except (RequestException, HTTPError) as e:
    raise RuntimeError(f"Failed to fetch access token: {e}")
except KeyError:
    raise RuntimeError("Access token not found in the response")

def load_users(url_vl, token, df=None):
    if df is None:
        df = pd.DataFrame()

    if not token:
        raise ValueError("Token must be provided")

    headers = {
        'Authorization': f"Bearer {token}"
    }
    try:
        response = requests.get(url_vl, headers=headers)
        response.raise_for_status()
        response_js = js.loads(response.text)

        # Extract and append data to the DataFrame
        current_df = pd.DataFrame(response_js.get('value', []))
        df = pd.concat([df, current_df], ignore_index=True)

        # Recursively load next pages if available
        next_link = response_js.get("@odata.nextLink")
        if next_link:
            return load_users(next_link, token, df)

    except KeyError as e:
        print(f"Key error encountered: {e}")

    return df

# Fetching the members of a specific group
users_df = load_users(  url_vl="https://graph.microsoft.com/v1.0/groups/<group_id>/members", token=token)
print("RLS Group User Details:")
print(users_df)

# Fetch the userPrincipalName and usageLocation for each user
users_location_df = load_users(    url_vl="https://graph.microsoft.com/v1.0/users?$select=userPrincipalName,usageLocation", token=token)
print("User Locations Information:")
print(users_location_df)

# Merge the userPrincipalName and usageLocation data with the existing group member data
df_merged = pd.merge(users_df, users_location_df[['userPrincipalName', 'usageLocation']], on='userPrincipalName', how='left')
print("Final Dataframe")
print(df_merged)

# Insert the updated merged data back to Snowflake
print('Insert Started')
session.write_pandas(df_merged, schema='TEST_SCHEMA', table_name="RLS_USERS", overwrite=True, auto_create_table=True)
print('Insert Completed')
